# Índices Educación Superior Periodo 2005-2023

## I. Introducción
La educación es uno de los pilares de la sociedad, por tanto se hace interesante comprender cómo se relacionan algunos de sus indicadores y de que forma influyen unos con otros ¿existirá una relación entre el puntaje de corte de una carrera y su costo en arancel? ¿se pueden agrupar carreras y universidades según su información de matrícula?

Se posee una base de datos histórica de educación superior que cuenta con registros desde 2005 a 2023 y que se explotará a continuación.

## II. Exploración de datos

### Distribuciones importantes

En primer lugar, se decide contabilizar el porcentaje de registros correspondientes a carreras de pregrado y postgrado, resultando en un 83.8% y 16.2% respectivamente.

![Distribución de registros con respecto a la columna Pregrado/Posgrado](./outputs/dist.png)


Por otra parte, la distribución de estudiantes por regiones es la siguiente:

![Distribución de estudiantes por regiones](./outputs/count_students.png)

Aunque pudiese ser predecible, se evidencia una alta cantidad de estudiantes en la Región Metropolitana, seguida por Valparaíso y Biobío. 

En cuanto a la cantidad de estudiantes por tipo de carrera, se observa que la mayoría de los estudiantes se encuentran en carreras de tipo técnicas, seguidas por profesional con licenciatura y carreras profesionales.

![Distribución de estudiantes por tipo de carrera](./outputs/count_students_degree.png)

Agrupando las carreras por sector de conocimiento, se puede observar una gran presencia de estudiantes en carreras tecnológicas, seguidas por el área de administración y comercio. En tercer puesto, se encuentra el área de salud que es casi equiparada por el área de educación.

![Distribución de estudiantes por sector de conocimiento](./outputs/count_students_area.png)

Por el lado de las estadísticas relacionadas al ingreso a estas carreras se tiene el promedio del puntaje obtenido en las pruebas de matemáticas y lenguaje. 
Los resultados registrados entre los años 2005 y 2019 corresponden a la prueba de selección universitaria PSU, con un puntaje mínimo de 150 puntos y un máximo de 850 puntos. Los siguientes 3 años corresponden a resultados de la prueba de transición universitaria PTU, mientras que el último año registrado corresponde a la nueva prueba de acceso a la educación superior PAES, cuyo rango de puntajes va desde los 100 hasta los 1.000 puntos.

A continuación se observa la distribución de los puntajes, donde se puede apreciar que en el caso de la PSU la mayoría de los estudiantes obtienen puntajes entre 400 y 700 puntos en ambas pruebas (esto ya que la escala de cada una de las pruebas de la PSU estaba estandarizado, con una media de 500 puntos y una desviación estándar de 110).


![Distribución de puntajes PSU](./outputs/psu.jpeg)

![Distribución de puntajes PSU](./outputs/paes.jpeg)

### Análisis de las columnas

Desde un punto de vista más detallado, se analizarán las columnas de la base de datos para determinar cuáles son las más relevantes para el análisis y cuáles podrían ser descartadas. Entre las columnas más importantes se encuentran las siguientes:

* Año: Se observa que la base de datos cuenta con registros desde el año 2005 hasta el año 2023. Este atributo es numérico y el tipo de dato de esta columna es de tipo entero.
* Cód. Institución: Corresponde al código que identifica a la institución de educación superior, por lo que es un atributo numérico. El tipo de dato de esta columna es de tipo entero, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agregación de datos.
* Nombre Institución: Atributo cualitativo nominal que corresponde al nombre de la institución de educación superior. El tipo de dato de esta columna es de tipo string, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agregación de datos.
* Tipo Institución: Corresponde al tipo de institución de educación superior, lo que lo convierte en un atributo cualitativo. El tipo de dato de esta columna es de tipo string, no requiriendo ningún tipo de transformación, aunque al ser un atributo cualitativo podría ser ordinal y por tanto ser mapeado. Podría ser útil para realizar agrupaciones de datos.
* Clasificación1 a Clasificación6: Son atributos cualitativo y, al igual que el "Tipo Institución" se les puede considerar ordinal. El tipo de dato de esta columna es de tipo string.
Dado que podrían ser categorías importantes, se analizan sus datos para determinar a qué corresponden. 

  - Clasificación 2: Corresponde al tipo de universidad: si es CRUCH, Privada, Instituto. FFAA, etc.
  - Clasificación 1: Es una versión menos granular de la clasificación 2, por lo que se decide eliminarla.
  - Clasificación 3: Acreditación de la carrera (Acreditada, No Acreditada)
  - Clasificación 4: Si es una organización autónoma, cerrada, de supervisión, licenciamiento, etc.
  - Clasificación 5: Si está o no adscrita a gratuidad
  - Clasificación 6: Subsistema al que pertenece la carrera

* Nombre de la Sede: Atributo cualitativo nominal que corresponde al nombre de la sede de la institución de educación superior. El tipo de dato de esta columna es de tipo string, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agregación o agrupación de datos.
* Comuna donde se imparte la carrera o programa y Nombre Region: Atributos cualitativo nominales que corresponden al nombre de la comuna donde se imparte la carrera o programa y su región. El tipo de dato de estas columnas son de tipo string, no requiriendo ningún tipo de transformación. Podrían ser útil para realizar agregación o agrupación de datos.
* Orden Geográfico de la Región (Norte a Sur): Atributo cuantitativo que corresponde al número de la región, siendo un número entero. El tipo de dato de esta columna es de tipo entero, no requiriendo ningún tipo de transformación.
* Cód. Carrera: Corresponde al código que identifica a la carrera, por lo que es un atributo numérico. El tipo de dato de esta columna es de tipo entero, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agregación de datos.
* Carrera Genérica, Nombre Programa y Mención o Especialdad: Información cualitativa nominal que corresponde al nombre de la carrera o programa y su mención/especialidad en caso de contar con una. El tipo de dato de estas columnas es de tipo string.
* Horario: Atributo cualitativo nominal que corresponde al horario en que se imparte la carrera o programa (Diurno o Vespertino). El tipo de dato de esta columna es de tipo string. 
* Tipo Programa: Sus valores son Programa Regular y Programa Especial (por lo que es un atributo cualitativo nominal). El programa especial aplica para programas que permiten el perfeccionamiento en competencias específicas, por lo que su duración puede ser menor a la de una carrera con programa regular o tratarse de programas de formación de las fuerzas armadas. El tipo de dato de esta columna es de tipo string.
* Area Conocimiento: Atributo cualitativo nominal que corresponde al área de conocimiento de la carrera o programa. El tipo de dato de esta columna es de tipo string. Podría ser útil para realizar agrupación de datos.
* idgenerocarrera: Atributo cuantitativo que corresponde al código que identifica al género de la carrera. El tipo de dato de esta columna es de tipo entero.
* Tipo Carrera: Atributo cualitativo nominal que corresponde al tipo de carrera (Técnica, Profesional, Profesional con Licenciatura, etc.). El tipo de dato de esta columna es de tipo string. Podría ser útil para realizar agrupación de datos.
* IngresoDirecto: Atributo cualitativo nominal que corresponde a si la carrera o programa permite el ingreso directo o no. El tipo de dato de esta columna es de tipo string.
* Año Inicio Actividades: Atributo cuantitativo que corresponde al año en que la carrera o programa inició sus actividades. El tipo de dato de esta columna es de tipo entero.
* Nombre del Campus: Atributo cualitativo nominal que corresponde al nombre del campus donde se imparte la carrera o programa. El tipo de dato de esta columna es de tipo string. Podría ser útil para realizar agrupación o agregación de datos.
* Duración (en semestres): Atributo cuantitativo que corresponde a la duración de la carrera o programa en semestres. El tipo de dato de esta columna es de tipo entero.
* Cód. Campus y Cód. Sede: Corresponden a los códigos que identifican al campus donde se imparte la carrera o programa, por lo que es un atributo numérico. El tipo de dato de estas dos columnan es de tipo entero, no requiriendo ningún tipo de transformación. Podrían ser útiles para realizar agregación de datos.
* Título y Grado Académico: Atributo cualitativo nominal que corresponde al título y grado que otorga la carrera o programa. El tipo de dato de esta columna es de tipo string. Podría ser útil para realizar agrupación de datos.
* Máximo Puntaje, Promedio Puntaje y Mínimo Puntaje (promedio matemáticas y lenguaje): Atributos cuantitativos que corresponden al puntaje máximo y mínimo de ingreso a la carrera o programa, así como el promedio de estos. El tipo de dato de estas columnas es de tipo entero.
* Puntaje de corte, (primer y último seleccionado y promedio de la carrera): Atributos cuantitativos que corresponden al puntaje de corte de la carrera o programa, así como el primer y último seleccionado y el promedio de estos. El tipo de dato de estas columnas es de tipo entero.
* Máximo Puntaje, Promedio Puntaje y Mínimo Puntaje (NEM): Atributos cuantitativos que corresponden al puntaje máximo y mínimo de notas de enseñanza media de los seleccionados, así como el promedio de estos. El tipo de dato de estas columnas es de tipo entero.
* Máximo Puntaje, Promedio Puntaje y Mínimo Puntaje (Ranking): Similar al caso anterior, pero para el ranking de notas de enseñanza media. El tipo de dato de estas columnas es de tipo entero.

A partir de esto, se observa que hay columnas que no nos entregan información relevante, como el orden geografico de la región, el Código SIES o el idgenerocarrera. 

También la información referente a puntajes mínimos y máximos de ingreso es insuficiente (tienen una alta tasa de nulos), pero se cuenta con más valores para la columna de 'Promedio Puntaje (promedio matemáticas y lenguaje)' de las pruebas de ingreso. Las menciones o especialidades no están presentes en todas las carreras y la columna con el nombre del campus tiene una tasa de nulos del 57%, por lo que tampoco se considerarán en el experimento.

## III. Preguntas de interés
Preguntas a que se podrían resolver usando la minería de datos:
1. ¿Se puede predecir la participación femenina?
2. ¿Existen universidades con características comunes? 
3. ¿Se pueden agrupar instituciones/carreras usando estas características? ¿Acaso tienen perfiles similares de aranceles y puntajes de ingreso?

Resulta interesante construir un predictor de puntajes de corte creado con datos reales, y no con simples estimaciones, o analizar el perfil de las instituciones de acuerdo a las características que se puedan extraer a partir de los atributos de la base de datos, ya que mediante simple análisis visual de las bases no siempre es directa la relación entre características de miles de datos disponibles.

Estas 4 preguntas pueden ser catalogadas en 2 grupos: preguntas de predicción y preguntas de agrupamiento (clustering).

A partir del estudio inicial del dataset se cree que hay datos suficientes para responder el primer grupo de preguntas antes planteadas, por lo tanto, se decide buscar más información en otras fuentes de datos.

## IV. Propuesta de Metodología Experimental
Considerando que nuestras preguntas serán estudiadas en dos categorías, se propone las siguientes metodologías:

**Predicción:**
- Realizar preprocesamiento: Botar columnas irrelevantes, rellenar nulos, etc.
- Fijar una carrera/clase de carrera específica: Considerando la cantidad de carreras existentes, se propone fijar una carrera específica para realizar la predicción de puntaje de corte.
- Usar modelos de regresión para predecir el puntaje de corte anual: Se debe determinar qué tipo de regresión conviene usar en función de los datos: lineal, polinomial, etc.
- Evaluar el modelo y ver qué features son importantes

**Clustering:**
- Preprocesamiento: La etapa de preprocesamiento se divide en 6 pasos
    - Botar filas irrelevantes
    - Botar columnas irrelevantes
    - Botar columnas con muchos nulos
    - Imputar nulos
    - Mapear columnas cualitativas ordinales
    - Aplicar OneHotEncoder a columnas cualitativas nominales
    - Unir ambos conjunto de datos
- Determinar número óptimo de clusters
- Usar algoritmos de clustering: K-means, DBSCAN, etc.
- Reducción de dimensiones: Se puede usar PCA para reducir la dimensionalidad de los datos y así poder visualizarlos en 2D
- Evaluación y análisis: Se debe evaluar qué tan bien se agrupan los datos y analizar los grupos obtenidos para determinar si tienen sentido o no.

## V. Experimentación

### Predicción participación femenina

#### * Limpieza
Como se indicó anteriormente, la gran cantidad y variedad de carreras genera una dificultad para analizar los datos, por lo tanto se decide solo considerar las carreras del área de ingeniería pues se han hecho esfuerzos a nivel nacional por incrementar la participación femenina en estas carreras.

Se realiza la limpieza de datos en base a los objetivos que se tienen, por lo que en primera instancia eliminamos las columnas que no tienen mayor relevancia o que repiten información. Por un lado, hay variables relacionadas con las instituciones, en donde está el nombre, su ID, y distintas clasificaciones, de las cuales se escogen las que indican que tipo de establecimiento es, si está acreditada y si está adscrita a la gratuidad.

Por otro lado, existen variables referidas a localidades y nombre del campus, en donde, al no realizarse un análisis specifico por sector, solo basta saber en que región se encuentra.

Finalmente, hay variables sobre las carreras en si, como la mención, tipos de carreras, grados, tipo de moneda, costos, entre otras, de las cuales interesan el horario, tipo de carrera, el título y tipo de programa.

Con esos criterios anteriores, de las 60 columnas iniciales, solo son de primer interes 41 que corresponden a :
'Año','Cód. Institución','Tipo Institución','Clasificación2', 'Clasificación3','Clasificación5','Orden Geográfico de la Región (Norte aSur)','Cód. Carrera','Carrera Genérica','Nombre Programa','Horario','Tipo Programa','Tipo Carrera','Duración (en semestres)','Título','Máximo Puntaje (promedio matemáticas y lenguaje)', 'Promedio Puntaje (promedio matemáticas y lenguaje)','Mínimo Puntaje (promedio matemáticas y lenguaje)', 'Puntaje de corte (primer seleccionado)','Puntaje de corte (promedio de la carrera)','Puntaje de corte (último seleccionado)','Máximo Puntaje NEM', 'Promedio Puntaje NEM', 'Mínimo Puntaje NEM', 'Máximo Puntaje Ranking','Promedio Puntaje Ranking', 'Mínimo Puntaje Ranking','Nº Alumnos Ingreso Via PSU o PDT','Nº Alumnos Ingreso Otra Via','Valor de matrícula', 'Valor de arancel', 'Valor del Título', 'Vacantes', 'Matrícula primer año hombres', 'Matrícula primer año mujeres','Matrícula primer año extranjeros','Matrícula total hombres','Matrícula Primer Año', 'Matrícula total mujeres', 'Matrícula total extranjeros','Matrícula Total'.

Luego de este primer filtro, es necesario realizar una mayor limpieza. Como el objetivo es analizar la matrícula de mujeres en el primer año, es necesario que sean carreras de programa regular. En particular nos interesan las universidades y considerando que se implementa una nueva escala de puntajes en la PAES tambien es necesario filtrar años. Es por esto que se eliminan los datos que no tengan un programa regular, que no sean universidades y que el año sea anterior al 2023.

Se observa que hay varias universidades que imparten un plan común para ingeniería en donde después de cierto tiempo se especializan, por lo que no existen matriculas de primer año en esas especializaciones. Como existe el dato de ingreso al plan común, solo nos quedamos con estos, eliminando todas las carreras que no tengan ingreso de mujeres en su primer año.

En la misma línea de matriculas, hay carreras que en ciertos años solo ingresaron hombres o solo ingresaron mujeres, por lo que fue necesario rellenar estos datos nulos con la resta de la matricula total y la matricula del genero que ingresa. Esto es posible pues la matricula de extranjeros no está contabilizada en el total de matriculas.

Una vez realizado todos los filtros anteriores, hay dos variables que se utilizaron para filtrar y que en este punto solo tienen un valor asociado, por lo qu para disminuir dimencionalidad se eliminan. Quedando un total de 39 columnas y 8084 filas.

#### * Modelo

### Clustering de carreras e instituciones
Como se mencionó anteriormente, el conjunto de datos inicial no parece ser suficiente para responder las preguntas de agrupamiento. Por lo tanto, se decide utilizar una base de datos con información de los inmuebles de las instituciones de educación superior y que contiene varios identificadores en común con el conjunto inicial, facilitando la unión de registros entre ambos.

Esta nueva base de datos posee diferentes pestañas, de las cuales se utilizarán las siguientes:
* Inmuebles: Contiene información sobre los inmuebles de las instituciones de educación superior, como la superficie construida, número de salas, etc. 
* Laboratorios y Talleres: Contiene información sobre la cantidad de laboratorios y talleres que posee cada institución de educación superior, incluyendo el número de computadores.
* Docentes: Esta pestaña contiene información sobre la cantidad de docentes que trabajan en cada institución de educación superior y el número de horas que trabajan, detallando también el número de docentes que trabajan en magíster y doctorado.

Con el fin de realizar los pasos propuestos en la metodología experimental, se procede a la creación de un pipeline de preprocesamiento. Esto, además de permitir ajustar los hiperparámetros de los algoritmos de preprocesamiento, evita el data leakage al momento de realizar la validación cruzada. 

![Ejemplo de pipeline](./outputs/pipeline_inicial.png)

Es importante mencionar que cada etapa del pipeline se realiza en cada uno de los grupos de datos por separado. Aunque pareciera más sencillo realizar la unión de los datos antes de realizar el preprocesamiento, el realizarlo de esta forma otorga más control en el proceso, pudiendo realizar experimentos con distintas combinaciones según el dataframe elegido. Además, para combatir la falta de eficiencia, se realiza el preprocesamiento de cada grupo de datos en paralelo usando el pipeline llamado MultiDataFramePipeline.

![Procesamiento realizado a la columna "Grado Académico" que finalmente fue botada](./outputs/procesamiento_grado.jpeg)

#### Pipeline del conjunto inicial
Partiendo por el conjunto inicial y recordando el desbalance de la base de datos, se decide utilizar solo las carreras de Pregrado (83.8% de los datos). También se botan las filas que no cuentan con información de su puntaje promedio en las pruebas de matemáticas y lenguaje, ya que esta columna viene a reemplazar a las columnas de puntajes máximos y mínimos que cuentan con una alta tasa de nulos. Estas dos acciones son realizadas fuera del pipeline.

El primer paso del pipeline de preprocesamiento del set inicial corresponde a la elimininación de columnas irrelevantes. Para esto, se eliminan las columnas que no aportan información relevante para el análisis, como el nombre de la sede del registro (existe Nombre del Campus que tiene una mejor tasa), el orden geográfico de la región, la mención o especialidad de la carrera, el idgenerocarrera, Código SIES y los puntajes que son innecesarias al usar la columna con su promedio. También se eliminan ciertas columnas para evitar la maldición de la dimensionalidad, como el grado académico (columna que fue procesada pero que se optó por no continuar para ser reemplazada por la columna "Tipo Carrera" que no contiene datos nulos), el código de la institución (debido a la existencia de la columna "Nombre Institución" que es más fácil de leer), el nombre de la región en que se imparte la carrera, la carrera genérica y el código de la carrera (debido a la existencia de la columna "Area de Conocimiento" que tiene una menor dimensionalidad), el nombre del programa, nombre del campus y título que otorga la carrera. El número de columnas pasa de 59 a 40 columnas.

![Detalle de un pipeline. Se observan las columnas a botar en el primer paso del pipeline](./outputs/pipeline_detalle.png)

Tras esto, se procede a obtener las columnas que cuentan con una alta tasa de nulos con la finalidad de eliminarlas. Se experimentó con un umbral de 0.24 y se decidió mantener la columna "Nombre del Campus" debido a que se considera como una columna importante para el análisis. Como resultado, se pasa de 40 a 34 columnas.

Una etapa adicional que se realiza es el de procesar el costo de la carrera/programa. Se estandariza la columna "Tipo Moneda" para obtener dos posibles valores: "Pesos" y "UF". Luego, para los registros cuyo valor de matrícula, arancel o título esta en UF, se multiplica por el valor de la UF del año actual. Finalmente, se elimina la columna "Tipo Moneda".

En el cuarto paso del pipeline se realiza el mapping de las columnas cualitativas ordinales (paso requerido por el modelo). El algoritmo mapea las categorías de una columna a valores numéricos, donde cada categoría es mapeada a un número entero. Por ejemplo, si una columna tiene las categorías "A", "B" y "C", estas serán mapeadas a 0, 1 y 2 respectivamente. Las columnas mapeadas son las Categorias del 1 al 6, Tipo Institución, Tipo Programa, Tipo Carrera, y Tipo Ingreso.

Ante la presencia de columnas cualitativas nominales, se decide utilizar el algoritmo OneHotEncoder para realizar el mapeo. Este algoritmo crea una columna por cada categoría de la columna original, donde cada columna creada corresponde a un valor binario que indica si la categoría está presente o no. Por ejemplo, si una columna tiene las categorías "A", "B" y "C", estas serán mapeadas a 3 columnas: "A", "B" y "C", donde cada columna tendrá un valor de 1 si la categoría está presente en el registro o 0 si no lo está. Las columnas mapeadas son las columnas "Nombre Institución", "Comuna donde se imparte la carrera o programa", "Area Conocimiento" y "Horario".

El último paso de este pipeline es el de imputar los nulos de las columnas restantes. Por el lado de los algoritmos de imputación univariante se encuentran el algoritmo SimpleImputer (que utiliza la media, mediana o moda para rellenar los nulos), mientras que por el lado de los multivariantes se está el algoritmo IterativeImputer (que utiliza un modelo de regresión para predecir los valores nulos). Adicionalmente, se cuenta con el algoritmo KNNImputer, que utiliza los k vecinos más cercanos para predecir los valores nulos. Debido a la falta de tiempo solo se experimenta con el algoritmo KNNImputer, elegido ya que considera la estructura local de los datos al imputar valores, lo que puede ser beneficioso en conjuntos de datos donde los vecinos cercanos son indicativos de relaciones relevantes. Se experimenta con distintos valores de k y se decide utilizar k=5, valor que computacionalmente no resulta demasiado costoso en este conjunto con gran cantidad de datos.

#### Pipeline del nuevo conjunto de datos
Pasando al pipeline de los conjuntos de datos "inmuebles", "labs" y "docencia", se realiza un preprocesamiento mucho más simple. En primer lugar, se eliminan las columnas que aparecen en el conjunto inicial puesto que no aportan más información (mantener estos datos generaría columnas duplicadas al momento de unir los datasets). Luego, se botan las columnas con alta tasa de nulos, usando un umbral de 0.3 y finalmente se imputan los nulos utilizando el algoritmo KNNImputer con k=5.

#### Pipeline de unión de los conjuntos de datos y ejecución de los algoritmos de clustering

Ejecutado cada pipeline en paralelo, se realiza la unión de los cuatro conjuntos de datos resultantes. Para esto, se realiza un join utilizando el código de la sede (identificador más particular encontrado) y el año del registro. Para evitar la reaparición de valores nulos, se une la intersección de los conjuntos de datos, reduciendo solo en un 15% la cantidad de registros del dataset. La ejecución de los pipeline en paralelo y la unión del resultado de estos se realiza en un nuevo pipeline.

El siguiente paso en este nuevo pipeline bota la columna "Cód. Sede" utilizada para hacer la unión. Esto, ya que se trata de un identificador en formato string (malo para el modelo) que ya no tiene utilidad.

Luego, como cuarto paso, se utiliza el algoritmo VarianceThreshold para eliminar las columnas con varianza menor a un umbral de 0.1. Esto, con el fin de eliminar columnas que no aportan información relevante.

Finalmente, antes de realizar los clusters, se estandarizan los datos utilizando el algoritmo StandardScaler. Esto, con el fin de evitar que los algoritmos de clustering sean sesgados por las diferencias de escala entre las columnas.

#### Generación del cluster
Para la generación de los grupos de datos se utilizan el algoritmo K-means. Este algoritmo es uno de los más utilizados para realizar clustering, ya que es fácil de implementar y es computacionalmente eficiente. Además, es un algoritmo que permite determinar el número de clusters a priori, lo que es una ventaja al momento de realizar el análisis.

Para determinar el número de clusters se utiliza el método del codo. Este método consiste en realizar el algoritmo de clustering para distintos valores de k y graficar el valor de la inercia (suma de las distancias al cuadrado de cada punto al centroide de su cluster) en función de k. El valor de k que se debe elegir corresponde al punto de la gráfica donde se observa un cambio de pendiente, lo que se conoce como el "codo" de la gráfica. En este caso, tras iterar el número de clusters desde 3 a 6, se observa que el codo se encuentra en k=4, por lo que se decide utilizar este valor para el algoritmo de clustering.

![Gráfica de la inercia en función de k](./outputs/codo.png)

#### Resultados

Como vista general, se presenta la gráfica en 2D obtenida mediante PCA.

![Gráfica de los clusters en 2D](./outputs/2d.png)

Si bien en esta gráfica se observa que los clusters se encuentran bien definidos (lo que es un buen indicador de que el algoritmo de clustering funcionó correctamente) hay que tener en cuenta que esta gráfica solo muestra 2 dimensiones de los datos, por lo que a continuación se presentan histogramas en busca de caracterizar los clusters obtenidos.

![Histograma de las instituciones por cluster](./outputs/nombre_institucion.jpeg)

![Histograma de los tipo de institución (Clasificación1) por cluster](./outputs/clasificacion1.jpeg)

![Histograma de los tipo de institución detallado(Clasificación2) por cluster](./outputs/clasificacion2.jpeg)

![Histograma del tipo de acreditación (Clasificación3) por cluster](./outputs/clasificacion3.jpeg)

![Histograma según adscripción a gratuidad (Clasificación5) por cluster](./outputs/clasificacion5.jpeg)

![Histograma según área de la carrera por cluster](./outputs/area.jpeg)

![Histograma según comuna en que se imparte la carrera por cluster](./outputs/comuna.jpeg)

![Histograma según tipo de carrera por cluster](./outputs/tipo.jpeg)

![Histograma según costo arancel por cluster](./outputs/valor_arancel.jpeg)

![Histograma según puntaje promedio matemáticas y lenguaje por cluster](./outputs/promedio_puntaje.jpeg)




# Anexo

In [ ]:
from pathlib import Path
import joblib

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline

from pipeline import (
    DropColumns,
    DropHighNAPercentage,
    NormalizeCurrency,
    OrdinalColumnMapper,
)

pd.set_option('display.max_columns', 100)

In [ ]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"
assert DATA_DIR.exists()

DATA_PATH = DATA_DIR / "base_indices_2005-2023.xlsx"

In [ ]:
# # xlsx to pkl
df = pd.read_excel(DATA_PATH)
joblib.dump(df, DATA_DIR / 'base_indices.pkl')

In [ ]:
df = joblib.load(DATA_DIR / "base_indices.pkl")
df = df.reset_index()

## Exploración

### Hito 1

In [ ]:
df.info()

In [ ]:
# Plot distribution of pregrado vs posgrado in a pie chart
conteo_pregrado = df[df['Pregrado/Posgrado'] == 'Pregrado'].shape[0]
conteo_posgrado = df[df['Pregrado/Posgrado'] == 'Posgrado'].shape[0]

# Crear un gráfico de sectores
labels = ['Pregrado', 'Posgrado']
sizes = [conteo_pregrado, conteo_posgrado]
colors = ['lightskyblue', 'lightcoral']

plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Distribución de Pregrado vs Posgrado')
plt.show()

In [ ]:
df['Nombre Region'].value_counts().plot(kind='barh')
plt.ylabel('Región')
plt.xlabel('Cantidad de estudiantes')
plt.title('Cantidad de estudiantes por región')

In [ ]:
df['Tipo Carrera'].value_counts().plot(kind='barh')
plt.ylabel('Tipo de carrera')
plt.xlabel('Cantidad de estudiantes')
plt.title('Cantidad de estudiantes por tipo de carrera')

In [ ]:
df['Area Conocimiento'].value_counts().plot(kind='barh')
plt.ylabel('Área de conocimiento')
plt.xlabel('Cantidad de estudiantes')
plt.title('Cantidad de estudiantes por área de conocimiento')

In [ ]:
df['Año'] = pd.to_datetime(df['Año'], format='%Y')
df.groupby("Año")['Promedio Puntaje (promedio matemáticas y lenguaje)'].max()

In [ ]:
# Datos PSU y PTU
datos_puntaje_PSU_PTU=df[df['Año'].dt.year<2023]['Promedio Puntaje (promedio matemáticas y lenguaje)'].value_counts().sort_index().plot()
plt.xlabel("Promedio puntaje")
plt.ylabel("Cantidad de estudiantes")
plt.title("Distribución de promedio de puntaje para la PSU-PTU de matemáticas y lenguaje")
plt.show()

# Datos PAES
datos_puntaje_PAES=df[(df['Año'].dt.year>=2023)]['Promedio Puntaje (promedio matemáticas y lenguaje)'].value_counts().sort_index().plot()
plt.xlabel("Promedio puntaje")
plt.ylabel("Cantidad de estudiantes")
plt.title("Distribución de promedio de puntaje para la PAES de matemáticas y lenguaje")

plt.show()

### Hito 2

In [ ]:
df['Orden Geográfico de la Región (Norte aSur)'].value_counts()

In [ ]:
# show percentage of null values in Mención o Especialidad
df['Mención o Especialidad'].isnull().sum() / df.shape[0]

In [ ]:
# show unique values in idgenerocarrera
df['idgenerocarrera'].value_counts()

In [ ]:

df['Códgo SIES'].value_counts()

In [ ]:
# show percentage of null values in each column and sort them in descending order
nulls = df.isnull().sum() / len(df) * 100
nulls.sort_values(ascending=False)


In [ ]:
interes1 = [
    "Clasificación1",
    "Clasificación2",
    "Clasificación3",
    "Clasificación4",
    "Clasificación5",
    "Clasificación6",
]
for column in interes1:
    print(f"Valores únicos de la columna {column}: \n \n{df[column].unique()} \n ")

interes2 = [
    "Horario",
    "Tipo Programa",
    "IngresoDirecto",
    "Tipo Moneda",
    "Pregrado/Posgrado",
]
for column in interes2:
    print(f"Valores únicos de la columna {column}: \n \n{df[column].unique() } \n ")


### Hito 3

#### Regresión

In [ ]:
df.info()

In [ ]:
# Fuciones auxiliares para la limpieza 

def nulos(dataframe):
  """
  Muestra la cantidad de nulos que tiene cada columna de un dataframe
  """
  nulos_por_columna = dataframe.isnull().sum()
  print(nulos_por_columna)

def unicos(dataframe):
  """
  Muestra la cantidad de valores únicos que tiene cada columna de un dataframe
  """
  unicos = dataframe.nunique()
  print(unicos)

def valoresunicos(dataframe,col):
  """
  Muestra los 10 primeros valores únicos que tiene una columna
  """
  valores = dataframe[col].unique()
  print(col)
  i=0
  for val in valores:
    if i <11:
      print(f'\t{val}')
      i+=1
    else:
      break

def rellenar_nulo_m(dataframe,columna1,columna2,columna3):
  """
  actualiza los valores de la columna 1 con la resta de las columnas 2 y 3
  """
  null = dataframe[dataframe[columna1].isnull()]
  null[columna1] = null[columna3]-null[columna2]

  dataframe.update(null)

In [ ]:
""""En primera instancia nos interesa ver la matrícula de mujeres en carreras de ingeniería, 
para esto solo nos quedamos con las carreras que contengan esta palabra clave"""

df_ing = df[df['Carrera Genérica'].str.contains('Ingeniería')]
nulos(df_ing)

In [ ]:
unicos(df_ing)

In [ ]:
"""Notemos que existen columnas que repiten información y otras que no son relevantes para nuestro enfoque, 
analizaremos un poco estas para decidir inicialmente que columnas conservar para después hacer una limpieza más profunda"""

# Información de la institución
valoresunicos(df_ing,"Tipo Institución")

Esta variable nos indica que tipo de institución es en la que se im parte la carrera, de esta podemos filtrar para enfocarnos en las universidades

In [ ]:
# Clasificaciones
valoresunicos(df_ing,"Clasificación1")
valoresunicos(df_ing,"Clasificación2")
valoresunicos(df_ing,"Clasificación3")
valoresunicos(df_ing,"Clasificación4")
valoresunicos(df_ing,"Clasificación5")
valoresunicos(df_ing,"Clasificación6")

Podemos ver que de estas variables:
* Clasificación1 es abarcada por Clasificación2, por lo que basta con quedarnos con la segunda pues entrega más información.
*Clasificación3 y Clasificación5 resultan interesantes analizar si impactan de alguna manera
* Clasificación4 y Clasificación6  no aporta información que queremos analizar

In [ ]:
# Lugares
valoresunicos(df_ing,"Nombre de la Sede")
valoresunicos(df_ing,"Comuna donde se imparte la carrera o programa")
valoresunicos(df_ing,"Nombre Region")
valoresunicos(df_ing,"Orden Geográfico de la Región (Norte aSur)")
valoresunicos(df_ing,"Nombre del Campus")

Notemos que como no queremos hacer un análisis de las matriculas por sector específicos, basta con quedarnos con la variable que entrega la región, para facilitar despues el procesamiento de datos, es conveniente dejar la variable que ya trae las regiones en número.

Por otro lado, el nombre que lleva el campus donde se imparte la carrera no entrega gran información a lo que buscamos.

In [ ]:
# Información Carrera
valoresunicos(df_ing,"Mención o Especialidad")
valoresunicos(df_ing,"Horario")
valoresunicos(df_ing,"Tipo Programa")
valoresunicos(df_ing,"Tipo Carrera")
valoresunicos(df_ing,"Título")
valoresunicos(df_ing,"Grado Académico")
valoresunicos(df_ing,"Tipo Moneda")

Como solo nos iteresa analizar las carreras de ing en forma general, es decir, sin indagar en su especialización u area esta variable no nos interesa.

La variable horario, Tipo Carrera y Título resultan ser interesantes de analizar

El tipo de programa nos va a permitir filtrar las carreras que no son de ingreso regular,es decir, podremos quedarnos con las que están enfocadas a estudiantes que ingresan por primera vez a la educación superior

Como los datos de valores de matricula, arancel y título están en pesos chilenos, la variable Tipo Moneda no nos interesa.

Finalmente, el grado académico no nos aporta información relevante, pues buscamos las matriculas en carreras de ingeniería que incluso puedan no tener un grado.


De esta forma de las 60 columnas iniciales del dataset, solo nos interesan 41

In [ ]:
columnas = ['Año','Cód. Institución','Tipo Institución','Clasificación2', 'Clasificación3','Clasificación5','Orden Geográfico de la Región (Norte aSur)','Cód. Carrera','Carrera Genérica','Nombre Programa','Horario',
           'Tipo Programa','Tipo Carrera','Duración (en semestres)','Título','Máximo Puntaje (promedio matemáticas y lenguaje)', 'Promedio Puntaje (promedio matemáticas y lenguaje)',
           'Mínimo Puntaje (promedio matemáticas y lenguaje)', 'Puntaje de corte (primer seleccionado)','Puntaje de corte (promedio de la carrera)','Puntaje de corte (último seleccionado)',
           'Máximo Puntaje NEM', 'Promedio Puntaje NEM', 'Mínimo Puntaje NEM', 'Máximo Puntaje Ranking','Promedio Puntaje Ranking', 'Mínimo Puntaje Ranking','Nº Alumnos Ingreso Via PSU o PDT',
           'Nº Alumnos Ingreso Otra Via','Valor de matrícula', 'Valor de arancel', 'Valor del Título', 'Vacantes', 'Matrícula primer año hombres', 'Matrícula primer año mujeres',
           'Matrícula primer año extranjeros','Matrícula total hombres','Matrícula Primer Año', 'Matrícula total mujeres', 'Matrícula total extranjeros','Matrícula Total']
df_ing["Año"] = df_ing["Año"].dt.year

ing = df_ing[columnas]
ing

Ahora, debemos hacer el filtro de instituciones, tipo de programa y años. Para ello queremos las carreras enfocadas en alumnos que ingresan por primera vez a la educación superior, en particular a Universidades que pidan puntaje para ingresar. Notemos además que los puntajes del año 2023 están a una escala distinta de los años anteriores, por lo que esta tendremos que eliminarla para no alterar resultados.

In [ ]:
ing = ing[(ing['Tipo Programa']=='Programa Regular')&(ing['Tipo Institución']=='Univ.')&(ing['Clasificación2']!='(d) Universidades Privadas')&(ing['Año']<2023)]
nulos(ing)

Nuestro objetivo es analizar la matricula de mujeres en primeros años, se sabe que en muchas universidades las carreras de ingeniería primero pasan por un plan común, por lo que cuando se especualizan no tienen matriculas en primer año, estas carreras debemos eliminarlas

In [ ]:
sin_matricula_primer = ing[(ing["Matrícula Primer Año"]==0)]

# eliminamos las carreras que no tienen matrículas en primer año
merged = pd.merge(ing, sin_matricula_primer, how='left', indicator=True)
ing_fin = merged[merged['_merge'] == 'left_only'].drop(columns='_merge')
nulos(ing_fin)

In [ ]:
"""Veamos que sucede con los nulos de matriculas de primer año de mujeres y hombres"""
ing_fin[(ing_fin["Matrícula primer año mujeres"].isnull()) | (ing_fin["Matrícula primer año hombres"].isnull())]

Como la matrícula total de primer año no considera la matricula de extranjeros, es facil ver que podemos rescatar el valor de estos nulos

In [ ]:
rellenar_nulo_m(ing_fin,"Matrícula primer año mujeres","Matrícula primer año hombres","Matrícula Primer Año")
rellenar_nulo_m(ing_fin,"Matrícula primer año hombres","Matrícula primer año mujeres","Matrícula Primer Año")

nulos(ing_fin)

In [ ]:
unicos(ing_fin)

In [ ]:
"""Como se realizaron filtros, ahora hay columnas que solo tienen 1 valor, por lo que combiene elimarlas y se procede a indicar los tipos de variables"""
ing_fin = ing_fin.drop(["Tipo Institución", "Tipo Programa"],axis=1)
ing_fin = ing_fin.astype({'Año': 'int64',
                    'Cód. Institución': 'category',
                    'Clasificación2': 'object',
                    'Clasificación3': 'object',
                    'Clasificación5': 'object',
                    'Orden Geográfico de la Región (Norte aSur)': 'int64',
                    'Cód. Carrera': 'category',
                    'Carrera Genérica': 'category',
                    'Nombre Programa': 'category',
                    'Horario': 'category',
                    'Tipo Carrera': 'category',
                    'Duración (en semestres)': 'float64',
                    'Título': 'category',
                    'Máximo Puntaje (promedio matemáticas y lenguaje)': 'float64',
                    'Promedio Puntaje (promedio matemáticas y lenguaje)': 'float64',
                    'Mínimo Puntaje (promedio matemáticas y lenguaje)': 'float64',
                    'Puntaje de corte (primer seleccionado)': 'float64',
                    'Puntaje de corte (promedio de la carrera)': 'float64',
                    'Puntaje de corte (último seleccionado)': 'float64',
                    'Máximo Puntaje NEM': 'float64',
                    'Promedio Puntaje NEM': 'float64',
                    'Mínimo Puntaje NEM': 'float64',
                    'Máximo Puntaje Ranking': 'float64',
                    'Promedio Puntaje Ranking': 'float64',
                    'Mínimo Puntaje Ranking': 'float64',
                    "Nº Alumnos Ingreso Via PSU o PDT": 'float64',
                    "Nº Alumnos Ingreso Otra Via": 'float64',
                    "Valor de matrícula": 'float64',
                    "Valor de arancel": 'float64',
                    "Valor del Título": 'float64',
                    "Vacantes": 'float64',
                    "Matrícula primer año hombres": 'float64',
                    "Matrícula primer año mujeres": 'float64',
                    "Matrícula primer año extranjeros": 'float64',
                    "Matrícula Primer Año": 'float64',
                    "Matrícula total hombres": 'float64',
                    "Matrícula total mujeres": 'float64',
                    "Matrícula total extranjeros": 'float64',
                    "Matrícula Total": 'float64'})